# Calculating Elastic Properties of a Rock

In [ ]:
import numpy as np

from drp_template.math import bound, GPa2Pa, Pa2GPa
from drp_template.image import plot_effective_modulus

## Task 1: Calculate the density, bulk, and shear modulus of the effective elastic medium
We consider a mixture of two minerals, namely quartz and clay, and we illustrate the estimation of the density, bulk and shear moduli of the effective solid medium. The clay type is illite. We assume that clay and quartz are present in the mixture with equal proportions, i.e. the volume of clay is equal to the volume of quartz, $V_\textrm{clay} = V_\textrm{quartz} = 0.5$. The elastic properties of quartz and clay can be found in the Table. 
1. Compute the density of the mixture.
2. Compute the bulk and shear moduli of the mixture (Voigt, Reuss, Voigt-Reuss-Hill).
3. Compute the bulk and shear moduli of the mixture (Hashin-Shtrikman).


|Mineral|Density $(\textrm{g}\,\textrm{cm}^{-1})$|Bulk modulus (GPa)|Shear modulus(GPa)|
|---|---|---|---|
|Quartz|2.65|36|45|
|Clay (illite)|2.55|21|7|
|Clay (kaolinite)|1.58|1.5|1.4|
|Calcite|2.71|76|32|
|Dolomite|2.87|95|45|



In [ ]:
from drp_template.math import density_solid_mix
import numpy as np

phases = ('Quartz', 'Calcite')
fractions = np.array([0.5, 0.5])
densities = np.array([2650, 2550])

rho_Mix = density_solid_mix(f_solid=fractions, rho_solid=densities)
print(f"The effective density of the quartz, calcite mixture is: {rho_Mix} (kg/m^3).")

In [ ]:
from drp_template.math import bound, GPa2Pa

bulk = np.array([36, 21])
# bulk = GPa2Pa(bulk)
shear = np.array([45, 7])
# shear = GPa2Pa(shear)

k_voigt, k_reuss, u_voigt, u_reuss, k_hill, u_hill = bound(f_volume=fractions, k_component=bulk,u_component=shear,type='voigt-reuss')
print('k_voigt-reuss (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u_voigt_reuss (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_hill and u_hill (Hill average for voigt-reuss):', [k_hill, u_hill])

In [ ]:
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(f_volume=fractions, k_component=bulk,u_component=shear,type='hashin-shtrikman')
print('k_hs (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u_hs (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

## Task 2: Calculate the elastic properties of the fluid components
The elastic properties of the fluid components (water, oil, and gas) are not constant and depend on reservoir conditions, such as temperature and pressure, as well as on the fluid composition and characteristics, such as water salinity, oil gravity, gas gravity, and gas–oil ratio (i.e., the amount of gas dissolved in oil). The density and the bulk modulus of the fluid components can be computed using empirical relations, namely the Batzle–Wang equations (Batzle and Wang 1992), that account for reservoir conditions and fluid characteristics. Table 2.2 shows possible ranges of the elastic properties of water, oil, and gas, for pressure values between 10 and 50 MPa and temperature values between 30º and 80ºC. For gas, we report the values of methane, which is the largest component of natural gas. Water is generally denser than oil, and oil is denser than gas. For this reason, gas is more compressible than oil, and oil is more compressible than water, which implies that the bulk modulus of water is higher than the bulk modulus of oil, and the bulk modulus of oil is higher than the bulk modulus of gas.

|Fluid|Density ()|Bulk modulus(GPa)|
|---|---|---|
|Water|0.95–1.15|2-3|
|Oil|0.65–0.95|0.8–1.8|
|Gas (methane)|0.05–0.25|0.01–0.15|

If the fluid saturations are known, then the density of the effective fluid phase can be calculated as the arithmetic average of the densities of the fluids weighted by their saturations.

In an isostress condition, we can assume that the fluid components are homogeneously mixed together. For a homogeneous mixture of fluids, the Reuss average provides the effective bulk modulus `k_reuss` of the fluid mixture. In a “patchy” saturation condition, where the fluid mixture is characterized by spatially variable saturations, the Voigt average provides an approximation of the effective bulk modulus `k_voigt` of the fluid mixture.


We consider a mixture of water and gas where the water saturation is s_water = 0.2 and the gas saturation is s_gas = 0.8. We assume fresh water with the following density and bulk modulus: rho_water = 1 g/cm3 and k_water = 2.25 GPa; and methane with the following properties: rhp_gas = 0.1 g/cm3 and k_gas = 0.1 GPa. 

1. Calculate the effective fluid density of the mixture.
2. Calculate the effective fluid bulk modulus for homogenous and patchy fluid mixture.

In [ ]:
from drp_template.math import density_fluid_mix, Brie_law

phases = ('Water', 'Gas')
saturation_fluid = np.array([0.2, 0.8])
density_fluid = np.array([1000, 100])
bulk_fluid = np.array([2.25, 0.1])

rho_fluid_mix = density_fluid_mix(s_fluid=saturation_fluid,rho_fluid=density_fluid)
print(f"The effective density of fluid mixture is: {rho_fluid_mix} (kg/m^3).")

In [ ]:
k_voigt, k_reuss = bound(f_volume=saturation_fluid, k_component=bulk_fluid,u_component=None,type='fluid')

k_Brie = Brie_law(s_water=0.2, s_oil=None, s_gas=0.8, k_water=2.25, k_oil=None, k_gas=0.1)

print(f"The effective fluid bulk modulus for homogenous fluid mixture is: {k_reuss} (GPa)")
print(f"The effective fluid bulk modulus for patchy fluid mixture is: {k_voigt} (GPa)")
print(f"The effective fluid bulk modulus for for patchy mixtures of water and hydrocarbon is: {k_Brie} (GPa, after Brie et al. 1995)")

## Task 3: Calculating Elastic Properties of a Rock for different porosities

In this rock analysis task, we have information about different mineral phases and their respective elastic properties. The phases considered are Quartz, Feldspar, Dolomite, and Calcite.

The bulk modulus of these minerals, representing their resistance to changes in volume under pressure, is provided. For Quartz, it is 36 GPa, for Feldspar 75 GPa, for Dolomite 95 GPa, and for Calcite 76 GPa. Similarly, the shear modulus, indicating the material's response to shear stress, is given as 45 GPa for Quartz, 25 GPa for Feldspar, 45 GPa for Dolomite, and 32 GPa for Calcite.

The fractional composition of these minerals in the rock is also known. The fractions are specified as 50% for Quartz, 30% for Feldspar, 10% for Dolomite, and 10% for Calcite.

Additionally, the rock has a porosity of 0.1, indicating the fraction of the rock's volume that is not filled with solid minerals.

1. What are the Voigt and Reuss boundaries?
2. Calculate the corresponding Hashin-Shtrikman bounds.
3. Plot the Voigt, Reuss, Voigt-Reuss-Hill and upper and lower Hashin-Shtrikman bounds in a Figure (Pa vs Fraction).

In [ ]:
import numpy as np

# The the constants
phase = ('Quartz', 'Feldspar', 'Dolomite', 'Calcite')

K_pore = 0.000142
K_Quartz = 36
K_Feldspar = 75
K_Dolomite = 95
K_Calcite = 76

U_pore = 0
U_Quartz = 45
U_Feldspar = 25
U_Dolomite = 45
U_Calcite = 32

f_Pore = np.array([0.1, 0.25])
f_Quartz = np.array([0.6, 0.6])
f_Feldspar = np.array([0.25, 0.25])
f_Dolomite = np.array([0.05, 0.05])
f_Calcite = np.array([0.1, 0.1])


f_Porosity = f_Pore
f_Solid = np.array([f_Quartz, f_Feldspar, f_Dolomite, f_Calcite])

print(f"Porosity:\n{f_Porosity}")
print("###########")
print(f"Solid:\n{f_Solid}")

Consider first to calculate only the solid fractions of the mixture according to your estimated porosity.
1. Calculate the fractions of the solid components for a porosity of 10%.
2. Calculate the fractions of the solid components for a porosity of 25%.

In [ ]:
from drp_template.math import get_normalized_f_solid

f_solid_norm = get_normalized_f_solid(porosity=f_Porosity, f_solid_components=f_Solid, type='solid')

print(f"{f_Porosity[0]} = {f_solid_norm[:,0]}")
print(f"{f_Porosity[1]} = {f_solid_norm[:,1]}")


Different way of notation:

In [ ]:
f_Pore = np.array([0.1, 0.25, 0.3])
f_Solid = np.array([[0.6, 0.25, 0.05, 0.1], [0.6, 0.25, 0.05, 0.1], [0.6, 0.25, 0.05, 0.1]]).T

f_solid_norm = get_normalized_f_solid(porosity=f_Porosity, f_solid_components=f_Solid, type='solid')

print(f"{f_Porosity[0]} = {f_solid_norm[:,0]}")
print(f"{f_Porosity[1]} = {f_solid_norm[:,1]}")
print(f"{f_Porosity[2]} = {f_solid_norm[:,2]}")

However, sometimes your fractions + porosity are equal to 1. In this case you can use `get_normalized_f_solid` with `type='solid-porosity'`.

In [6]:
f_Pore = np.array([0.1, 0.25, 0.3])
f_Solid = np.array([[0.5, 0.25, 0.05, 0.1], [0.55, 0.1, 0.05, 0.05], [0.5, 0.1, 0.05, 0.05]]).T

f_solid_norm = get_normalized_f_solid(porosity=f_Porosity, f_solid_components=f_Solid, type='solid-porosity')

print(f"{f_Porosity[0]} = {f_solid_norm[:,0]}")
print(f"{f_Porosity[1]} = {f_solid_norm[:,1]}")
print(f"{f_Porosity[2]} = {f_solid_norm[:,2]}")

0.1 = [0.5  0.25 0.05 0.1 ]
0.25 = [0.55 0.1  0.05 0.05]
0.3 = [0.5  0.1  0.05 0.05]


In [ ]:













bulk_minerals = np.array([K_Quartz, K_Feldspar, K_Dolomite, K_Calcite])
shear_minerals = np.array([U_Quartz, U_Feldspar, U_Dolomite, U_Calcite])
f_Solid = np.array([f_Quartz/(1-f_Pore), f_Feldspar/(1-f_Pore), f_Dolomite/(1-f_Pore), f_Calcite/(1-f_Pore)])

print(f"f_Solid: {f_Solid} | sum: {round(np.sum(f_Solid),15)}")

# Calculate the Voigt and Reuss boundaries
k_voigt, k_reuss, u_voigt, u_reuss, k_hill, u_hill = bound(type='voigt-reuss', f_volume=f_Solid, k_component=bulk_minerals, u_component=shear_minerals)

print('k_voigt-reuss (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u_voigt_reuss (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_hill and u_hill (Hill average for voigt-reuss):', [k_hill, u_hill])

# Calculate the Hashin-Shtrikman boundaries
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', f_volume=f_Solid, k_component=bulk_minerals, u_component=shear_minerals)

print('k_hs (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u_hs (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

In [ ]:
# The the constants
phase = ('Pore', 'Quartz', 'Feldspar', 'Dolomite', 'Calcite')

K_Pore = 0.000142
K_Quartz = 36
K_Feldspar = 75
K_Dolomite = 95
K_Calcite = 76

U_Pore = 0
U_Quartz = 45
U_Feldspar = 25
U_Dolomite = 45
U_Calcite = 32

f_Pore = 0.2
f_Quartz = 0.3
f_Feldspar = 0.3
f_Dolomite = 0.1
f_Calcite = 0.1

bulk_phases = np.array([K_Pore, K_Quartz, K_Feldspar, K_Dolomite, K_Calcite])
shear_phases = np.array([U_Pore, U_Quartz, U_Feldspar, U_Dolomite, U_Calcite])
f_Rock = np.array([f_Pore, f_Quartz, f_Feldspar, f_Dolomite, f_Calcite])

print(f"f_Rock: {f_Solid} | sum: {round(np.sum(f_Rock),15)}")

# Calculate the Voigt and Reuss boundaries
k_voigt, k_reuss, u_voigt, u_reuss, k_hill, u_hill = bound(type='voigt-reuss', f_volume=f_Rock, k_component=bulk_phases, u_component=shear_phases)

print('k_voigt-reuss (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u_voigt_reuss (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_hill and u_hill (Hill average for voigt-reuss):', [k_hill, u_hill])

# Calculate the Hashin-Shtrikman boundaries
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', f_volume=f_Rock, k_component=bulk_phases, u_component=shear_phases)

print('k_hs (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u_hs (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

def bound2(type, f_solid, k_solid, u_solid):
    """
    
    """
# Ensure f_solid is a 2D array
    if f_solid.ndim == 1:
        f_solid = f_solid[:, np.newaxis]

    if len(f_solid) != len(k_solid) != len(u_solid):
        raise ValueError('Input fractions, k, and u must have the same length')

    c = 4 / 3
    
    num_components, num_samples = f_solid.shape
    k_voigt = np.zeros((num_components, num_samples))
    k_reuss = np.zeros((num_components, num_samples))
    u_voigt = np.zeros((num_components, num_samples))
    u_reuss = np.zeros((num_components, num_samples))
    k_avg = np.zeros((num_components, num_samples))
    u_avg = np.zeros((num_components, num_samples))

    for i in range(num_samples):  # Loop over samples
        if type == 'voigt-reuss':  # Voigt-Reuss bounds
            # Voigt (upper) bound for bulk modulus for each phase
            k_voigt[:, i] = f_solid[:, i] * k_solid
            print(f"k_voigt: {k_voigt}")

            # Reuss (lower) bound for bulk modulus
            k_reuss[:, i] = 1 / np.sum(f_solid[:, i] / k_solid)  # Accumulate results

            # Voigt (upper) bound for shear modulus
            u_voigt[:, i] = np.sum(f_solid[:, i] * u_solid)  # Accumulate results

            # Reuss (lower) bound for shear modulus
            u_reuss[:, i] = 1 / np.sum(f_solid[:, i] / u_solid)  # Accumulate results

            # Hill average for bulk and shear moduli
            k_avg[:, i] = (k_voigt[:, i] + k_reuss[:, i]) / 2  # Accumulate results
            u_avg[:, i] = (u_voigt[:, i] + u_reuss[:, i]) / 2  # Accumulate results
        
        elif type == 'hashin-shtrikman':  # Hashin-Shtrikman bounds
            kmx, kmn = np.max(k_solid), np.min(k_solid)
            umx, umn = np.max(u_solid), np.min(u_solid)
    
            # HS upper bound for bulk modulus
            k_hs_upper = 1 / (f_solid[:, i] / (k_solid + c * umx)) - c * umx
            # HS lower bound for bulk modulus
            k_hs_lower = 1 / (f_solid[:, i] / (k_solid + c * umn)) - c * umn
    
            etamx = umx * (9 * kmx + 8 * umx) / (kmx + 2 * umx) / 6
            etamn = umn * (9 * kmn + 8 * umn) / (kmn + 2 * umn) / 6
    
            # HS upper bound for shear modulus
            u_hs_upper = 1 / (f_solid[:, i] / (u_solid + etamx)) - etamx
            # HS lower bound for shear modulus
            u_hs_lower = 1 / (f_solid[:, i] / (u_solid + etamn)) - etamn
    
            # Simple arithmetic average for bulk and shear moduli
            k_avg[:, i] = (k_hs_upper + k_hs_lower) / 2
            u_avg[:, i] = (u_hs_upper + u_hs_lower) / 2
            
    # Sum over the phases
    k_voigt = np.sum(k_voigt, axis=0)

    return k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg

def plot_effective_modulus2(phi, modulus, k_voigt, dark_mode=False, fig_width=8, fig_height=6):
    """
    Plot the Voigt bound for effective modulus.

    Parameters:
    - phi (numpy.ndarray): Porosity values.
    - modulus (str): Type of modulus ('bulk' or 'shear').
    - k_voigt (numpy.ndarray): Voigt bound for bulk or shear modulus with shape (num_chunks, num_points).
    - dark_mode (bool): If True, use dark mode colors.
    - fig_width (int): Width of the figure.
    - fig_height (int): Height of the figure.

    Returns:
    - fig, ax: Figure and axis objects.
    """
    if dark_mode:
        text_color, face_color, edge_color = 'white', 'black', 'white'
    else:
        text_color, face_color, edge_color = 'black', 'white', 'black'

    # Define the range of porosity values to calculate
    phi_all = np.linspace(0, 1, 100)

    fig, ax = plt.subplots(figsize=(fig_width, fig_height), facecolor=face_color, edgecolor=edge_color)
    fig.set_facecolor(face_color)

    title = f"Effective {modulus.capitalize()} Modulus - Voigt Bound"
    ax.set_title(title, color=text_color)


    ax.plot(phi, k_voigt, label="Voigt Upper Bound")
    # ax.plot(phi, K_Reuss, label="Reuss Lower Bound")
    # ax.plot(phi, K_Voigt_Reuss_Hill, label="Voigt-Reuss-Hill")
    # ax.plot(phi, K_up_HS, label="Hashin–Shtrikman upper", color="brown", linestyle="dashed")
    # ax.plot(phi, K_low_HS9, label="Hashin–Shtrikman lower", color="brown", linestyle="-.")

    ax.set_xlabel("Porosity", color=text_color)
    ax.set_ylabel(f"{modulus.capitalize()} Modulus ($GPa$)", color=text_color)

    # Set the face and edge color of the axes (background within the plot)
    ax.set_facecolor(face_color)
    for spine in ax.spines.values():
        spine.set_edgecolor(edge_color)

    legend = ax.legend(facecolor=face_color, edgecolor=edge_color)

    for text in legend.get_texts():
        text.set_color(text_color)

    ax.tick_params(axis='y', colors=text_color, which='both')
    ax.tick_params(axis='x', colors=text_color, which='both')
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.00))  # convert x-axis into percent
    plt.xlim([0, 0.5])  # Set the x-axis limit to 50%
    plt.ylim([0, 100])  # Set the x-axis limit to 50%

    return fig, ax


from decimal import Decimal, getcontext

# The the constants
phase = ('Pore', 'Quartz', 'Feldspar', 'Dolomite', 'Calcite')

# Set the precision to a higher value
getcontext().prec = 20  # Adjust the precision as needed

K_Pore = 0.000142
K_Quartz = 36
K_Feldspar = 75
K_Dolomite = 95
K_Calcite = 76

U_Pore = K_Pore
U_Quartz = 45
U_Feldspar = 25
U_Dolomite = 45
U_Calcite = 32

# Define the range of porosity values to calculate
f_Pore = np.linspace(0, 1, 100)
# f_Pore = 0.2
f_Quartz = 0.3
f_Feldspar = 0.3
f_Dolomite = 0.1
f_Calcite = 0.1

bulk_phases = np.array([K_Pore, K_Quartz, K_Feldspar, K_Dolomite, K_Calcite])
shear_phases = np.array([U_Pore, U_Quartz, U_Feldspar, U_Dolomite, U_Calcite])

# Initialize an array to store the solid fractions
f_Rock = np.zeros((len(phase), len(f_Pore)))

# Calculate the corresponding solid fractions for each mineral
for i in range(len(f_Pore)):
    f_Rock[:, i] = np.array([
        f_Pore[i],
        f_Quartz / (1 - f_Pore[i]),
        f_Feldspar / (1 - f_Pore[i]),
        f_Dolomite / (1 - f_Pore[i]),
        f_Calcite / (1 - f_Pore[i])
    ])

# # Calculate the Voigt and Reuss boundaries
k_voigt, k_reuss, u_voigt, u_reuss, k_hill, u_hill = bound2(type='voigt-reuss', f_solid=f_Rock, k_solid=bulk_phases, u_solid=shear_phases)


print(f"k_voigt shape: {np.shape(k_voigt)}")
print(f"k_voigt: {k_voigt}")


phi = f_Pore
modulus_data = {
    'bulk': {
        'voigt': k_voigt,  # List of arrays for different cases
        'reuss': k_reuss,
        # 'hs_upper': [your_k_hs_upper_array_1, your_k_hs_upper_array_2, ...],
        # 'hs_lower': [your_k_hs_lower_array_1, your_k_hs_lower_array_2, ...],
        'avg': k_hill
    },
    'shear': {
        'voigt': u_voigt,
        'reuss': u_reuss,
        # 'hs_upper': [your_u_hs_upper_array_1, your_u_hs_upper_array_2, ...],
        # 'hs_lower': [your_u_hs_lower_array_1, your_u_hs_lower_array_2, ...],
        'avg': u_hill,
    }
}

fig, ax = plot_effective_modulus2(phi, modulus='bulk', k_voigt=k_voigt, dark_mode=False, fig_width=8, fig_height=6)


# print('k_voigt-reuss (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
# print('u_voigt_reuss (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
# print('k_hill and u_hill (Hill average for voigt-reuss):', [k_hill, u_hill])

# # Calculate the Hashin-Shtrikman boundaries
# k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', f_solid=f_Solid, k_solid=bulk_minerals, u_solid=shear_minerals)

# print('k_hs (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
# print('u_hs (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
# print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

In [ ]:
# Plot the elastic rock properties (Pa vs Fraction plot)
# Convert GPa in Pa
# k_voigt, k_reuss, u_voigt, u_reuss = GPa2Pa(k_voigt, k_reuss, u_voigt, u_reuss)
# k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = GPa2Pa(k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg)



data_bulk = {
    'voigt': k_voigt, 
    'reuss': k_reuss,
    'hs_upper': k_hs_upper,
    'hs_lower': k_hs_lower,
    'avg': k_avg
}

data_shear = {
    'voigt': u_voigt, 
    'reuss': u_reuss,
    'hs_upper': u_hs_upper,
    'hs_lower': u_hs_lower,
    'avg': u_avg
}


plot_effective_modulus(phi=f_Pore, modulus='bulk', data=data_bulk, types=['voigt','reuss','hs_upper', 'hs_lower'], dark_mode=False)

In [ ]:
from drp_template.image import plot_effective_modulus2
from drp_template.math import bound2

data_combined = {
    'bulk': {
        'voigt': k_voigt, 
        'reuss': k_reuss,
        'hs_upper': k_hs_upper,
        'hs_lower': k_hs_lower,
        'avg': k_avg
    },
    'shear': {
        'voigt': u_voigt, 
        'reuss': u_reuss,
        'hs_upper': u_hs_upper,
        'hs_lower': u_hs_lower,
        'avg': u_avg
    }
}

phi_all = np.linspace(0, 1, 100)
fractions_long = np.array([phi_all, (1-phi_all)*f_Quartz, (1-phi_all)*f_Feldspar, (1-phi_all)*f_Dolomite, (1-phi_all)*f_Calcite])
fractions_short = np.array([porosity, (1-porosity)*f_Quartz, (1-porosity)*f_Feldspar, (1-porosity)*f_Dolomite, (1-porosity)*f_Calcite])
# print(fractions[:, 2])
print(np.size(fractions_long))
print(np.size(fractions_short))


# # Calculate individual mineral contributions
# fraction_Quartz = phi_all
# fraction_Feldspar = (1 - phi_all) * f_Feldspar
# fraction_Dolomite = (1 - phi_all) * f_Dolomite
# fraction_Calcite = (1 - phi_all) * f_Calcite

# # Sum up contributions to get the total fraction for each point
# fractions = fraction_Quartz + fraction_Feldspar + fraction_Dolomite + fraction_Calcite
# print(fractions)

test = bound2(type='voigt-reuss', fractions=fractions_long, k=bulk_minerals, u=shear_minerals)

print(np.size(test))


plot_effective_modulus2(fractions_long, modulus='bulk', k=test, u=shear_minerals, data=data_combined, types=['hs_lower', 'hs_upper'], dark_mode=True)

In [ ]:
import numpy as np
from drp_template.math import bound, GPa2Pa, Pa2GPa
from drp_template.image import plot_effective_modulus2

# The the constants
phase = ('Quartz', 'Feldspar', 'Dolomite', 'Calcite')

K_Pore = 1e-20
K_Quartz = 36
K_Feldspar = 75
K_Dolomite = 95
K_Calcite = 76

U_Pore = 1e-20
U_Quartz = 45
U_Feldspar = 25
U_Dolomite = 45
U_Calcite = 32

f_Pore = 0.1
f_Quartz = 0.5
f_Feldspar = 0.3
f_Dolomite = 0.1
f_Calcite = 0.1

bulk_minerals = np.array([K_Quartz, K_Feldspar, K_Dolomite, K_Calcite])
shear_minerals = np.array([U_Quartz, U_Feldspar, U_Dolomite, U_Calcite])
fractions = np.array([f_Quartz, f_Feldspar, f_Dolomite, f_Calcite])



# Calculate the Voigt and Reuss boundaries
k_voigt, k_reuss, u_voigt, u_reuss, k_hill, u_hill = bound(type='voigt-reuss', f_volume=fractions, k_component=bulk_minerals, u_component=shear_minerals)

print('k_bulk (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_avg and u_avg (Hill average for voigt-reuss):', [k_hill, u_hill])


# Calculate the Hashin-Shtrikman boundaries
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', f_volume=fractions, k_component=bulk_minerals, u_component=shear_minerals)

print('k_bulk (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

data = {
    'voigt': k_voigt,
    'reuss': k_reuss,
    'hs_upper': k_hs_upper,
    'hs_lower': k_hs_lower,
    'avg': k_hill
    }

plot_effective_modulus(phi=fractions, data=data, modulus='bulk', types='voigt', dark_mode=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from drp_template.image import plot_effective_modulus2
from drp_template.math import bound2

# Given data
K_pore = 0
K_quartz = 36
K_Feldspar = 75
K_Dolomite = 95
K_Calcite = 76

U_pore = 0
U_quartz = 45
U_Feldspar = 25
U_Dolomite = 45
U_Calcite = 32

f_Pore = 0
f_Quartz = 0.5
f_Feldspar = 0.3
f_Dolomite = 0.1
f_Calcite = 0.1

# Bulk and shear modulus values for different minerals
bulk_minerals = np.array([K_pore, K_quartz, K_Feldspar, K_Dolomite, K_Calcite])
shear_minerals = np.array([U_pore, U_quartz, U_Feldspar, U_Dolomite, U_Calcite])

# Porosity values
phi_data = np.linspace(0, 1, 100)

# Calculate bounds using bound2 function
k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg = bound2(type='voigt-reuss', fractions=phi_data, k=bulk_minerals, u=shear_minerals)
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg_hs, u_avg_hs = bound2(type='hashin-shtrikman', fractions=phi_data, k=bulk_minerals, u=shear_minerals)

data = {
    'bulk': {
        'voigt': k_voigt,
        'reuss': k_reuss,
        'hs_upper': k_hs_upper,
        'hs_lower': k_hs_lower,
        'avg': k_avg_hs
    },
    'shear': {
        'voigt': u_voigt,
        'reuss': u_reuss,
        'hs_upper': u_hs_upper,
        'hs_lower': u_hs_lower,
        'avg': u_avg_hs
    }
}



# Example usage of plot_effective_modulus2 with bound2 output
fig, ax = plot_effective_modulus2(phi=phi_data,
                                  modulus='bulk',
                                  data=data,
                                  k=bulk_minerals,
                                  u=shear_minerals,
                                  types=['voigt', 'reuss', 'hs_upper', 'hs_lower', 'avg'],
                                  dark_mode=False)

plt.show()


## Task 2
In this material analysis task, we have information about different components and their corresponding elastic properties. Specifically, we are examining Quartz, Calcite, and Water.

For Quartz, the bulk modulus (K) is 36 GPa, and the shear modulus (U) is 45 GPa. The fractional composition of Quartz in the material is 80%.

For Calcite, the bulk modulus is 75 GPa, and the shear modulus is 31 GPa. The fractional composition of Calcite in the material is 20%.

Additionally, Water is considered as part of the material. The bulk modulus of Water is 2.2 GPa, and the shear modulus is practically negligible (close to 0). The fractional composition of Water in the material is 27%, which is equivalent to the porosity of the material.

The overall porosity of the material is specified as 27%.

Calculate the Voigt-Reuss or Hashin-Shtrikman bounds for 

In [ ]:
K_quartz = 36 # GPa
U_quartz = 45 # GPa
f_Quartz = 0.8
K_calcite = 75 # GPa
U_calcite = 31 # GPa
f_calcite = 0.2
K_water = 2.2 # GPa
U_water = 1e-20# GPa
f_water = 0.27 # f_water is equal to porosity
porosity = f_water

fractions = np.array([porosity, (1-porosity)*f_Quartz, (1-porosity)*f_calcite])
bulk_array = GPa2Pa(np.array([K_water, K_quartz, K_calcite]))
shear_array = GPa2Pa(np.array([U_water, U_quartz, U_calcite]))

# fractions = np.array([f_quartz, f_calcite])
# bulk_array = np.array([K_quartz, K_calcite])
# shear_array = np.array([U_quartz, U_calcite])

print(f"fractions: {fractions}")
print(bulk_array)

Calculate the Voigt (upper) and Reuss (lower) bounds.

In [ ]:
k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg = bound(type='voigt-reuss', f_volume=fractions, k_component=bulk_array, u_component=shear_array)

# convert Pa to GPa
k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg = Pa2GPa(k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg)
print('k_bulk (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_avg and u_avg (Hill average for voigt-reuss):', [k_avg, u_avg])

Calculate the upper and lower Hashin-Shtrikman bounds.

In [ ]:
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', f_volume=fractions, k_component=bulk_array, u_component=shear_array)

# convert Pa to GPa
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = Pa2GPa(k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg)
print('k_bulk (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

## Trial-And-Error

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

K_Reuss = 49
K_Voigt = 57.6
K_Voigt_Reuss_Hill = 53
K_up_HS = 5.3
K_low_HS = 5.2
phi = 0.2
# K_Reuss = k_reuss
# K_Voigt = k_voigt
# K_Voigt_Reuss_Hill = k_avg
# K_up_HS = k_hs_upper
# K_low_HS = k_hs_lower

fig, ax1 = plt.subplots()
color1 = 'peru'
color2 = 'tomato'
ax1.set_title("Effective Bulk Modulus ($GPa$)")
ax1.plot(phi, K_Reuss, label="Reuss Lower Bound")
ax1.plot(phi, K_Voigt, label="Voigt Upper Bound")
ax1.plot(phi, K_Voigt_Reuss_Hill, label="Voigt-Reuss-Hill")
ax1.plot(phi, K_up_HS, label="Hashin–Shtrikman upper", color="brown", linestyle="dashed")
ax1.plot(phi, K_low_HS, label="Hashin–Shtrikman lower", color="brown", linestyle="-.")

ax2 = ax1.twinx()
ax2.plot(phi, K_low_HS, label="Hashin–Shtrikman lower", color="brown", linestyle="-.")
ax2.set_ylabel("Clay", color=color2)
ax1.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Place legend outside the plot
ax1.tick_params(axis='y', colors=color1)
ax2.spines['left'].set_color(color1)
ax1.set_xlabel("Clay content")
ax1.set_ylabel("Quartz", color=color1)
ax2.tick_params(axis='y', colors=color2)
ax2.spines['right'].set_color(color2)

ax1.xaxis.set_major_formatter(mtick.PercentFormatter(1.00))  # convert x-axis into percent
plt.xlim([0, 1])

plt.show()
